In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [32]:
full_data = pd.read_csv('f1_data.csv')
# relevant_data = full_data.drop(["has_fastest_lap","race_laps_completed","points","grid_position", "fastest_lap_position"], axis = 1)
relevant_data = full_data.drop(["has_fastest_lap","race_laps_completed","points","grid_position","fastest_lap_position", "fp1_position","fp2_position","fp3_position"], axis = 1)
relevant_data = relevant_data.iloc[14744:]

label_encoder = LabelEncoder()
relevant_data['position_encoded'] = label_encoder.fit_transform(relevant_data['race_finishing_position'])
print(relevant_data['position_encoded'])

# Split the data into training and testing sets
X = relevant_data.drop(['race_finishing_position', 'position_encoded'], axis=1).values
y = to_categorical(relevant_data['position_encoded'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Create the neural network model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X.shape[1],)))
# model.add(BatchNormalization())  # Batch normalization layer
model.add(Dropout(0.2))  # Dropout layer
model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())  # Batch normalization layer
model.add(Dropout(0.2))  # Dropout layer
model.add(Dense(20, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)  # Set the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Make predictions
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
predicted_positions = label_encoder.inverse_transform(predicted_labels)


Epoch 1/50


ValueError: in user code:

    File "/Users/anirudhkrishna/GitHub/FormulaData/formulavenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/anirudhkrishna/GitHub/FormulaData/formulavenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/anirudhkrishna/GitHub/FormulaData/formulavenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/anirudhkrishna/GitHub/FormulaData/formulavenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/anirudhkrishna/GitHub/FormulaData/formulavenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1139, in compute_

In [29]:
for prediction in predictions[0:5]:
    print(np.max(prediction))

0.09907185
0.09907185
0.09907185
0.09907185
0.09907185
